# Optuna

ハイパーパラメータ調整ツール「Optuna」

https://github.com/optuna/optuna

In [1]:
!pip install -U optuna keras

     |████████████████████████████████| 163kB 7.0MB/s 
     |████████████████████████████████| 378kB 35.5MB/s 
     |████████████████████████████████| 1.1MB 43.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 112kB 35.0MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=27da7d7e7fd67308e7379ca46f2c5fef70ed7f8827d645db166c207c24a5d2b5
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.3.0-cp36-none-any.whl size=221121 sha256=c0bd37a986272eb3ef24d83e633b58b6f82b4e929bd9948fd0f997002c5d

In [2]:
from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop

import optuna

Using TensorFlow backend.


In [0]:
N_TRAIN_EXAMPLES = 3000
N_TEST_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10

In [0]:
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_test = x_test.reshape(-1, img_x, img_y, 1)[:N_TEST_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_test = y_test[:N_TEST_EXAMPLES]
    input_shape = (img_x, img_y, 1)

    model = Sequential()
    model.add(
        Conv2D(
            filters=trial.suggest_categorical("filters", [32, 64]),
            kernel_size=trial.suggest_categorical("kernel_size", [3, 5]),
            strides=trial.suggest_categorical("strides", [1, 2]),
            activation=trial.suggest_categorical("activation", ["relu", "linear"]),
            input_shape=input_shape,
        )
    )
    model.add(Flatten())
    model.add(Dense(CLASSES, activation="softmax"))

    # We compile our model with a sampled learning rate.
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_test, y_test),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the test set.
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]

In [5]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

11493376/11490434 [==============================] - 1s 0us/step


[I 2020-04-04 13:36:41,794] Finished trial#0 with value: 0.8870000243186951 with parameters: {'filters': 32, 'kernel_size': 5, 'strides': 2, 'activation': 'relu', 'lr': 0.0002951612857808553}. Best is trial#0 with value: 0.8870000243186951.
[I 2020-04-04 13:36:43,938] Finished trial#1 with value: 0.5699999928474426 with parameters: {'filters': 32, 'kernel_size': 5, 'strides': 2, 'activation': 'relu', 'lr': 1.8979047788078015e-05}. Best is trial#0 with value: 0.8870000243186951.
[I 2020-04-04 13:36:47,134] Finished trial#2 with value: 0.9300000071525574 with parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu', 'lr': 0.0006202429234253826}. Best is trial#2 with value: 0.9300000071525574.
[I 2020-04-04 13:36:49,652] Finished trial#3 with value: 0.6159999966621399 with parameters: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'linear', 'lr': 0.010623602674379158}. Best is trial#2 with value: 0.9300000071525574.
[I 2020-04-04 13:36:52,882] Finish

Number of finished trials: 100
Best trial:
  Value: 0.9610000252723694
  Params: 
    filters: 64
    kernel_size: 5
    strides: 1
    activation: relu
    lr: 0.007967644983944562
